In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import yfinance as yf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi import NewsApiClient

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from fuzzywuzzy import process
import requests

In [3]:
# Retrieve the News API key
news_api = os.getenv("NEWS_API_KEY")

# Create the newsapi client
newsapi = NewsApiClient(api_key=news_api)

In [4]:
company_request = 'columbia'

In [5]:
def getCompany(text):
    r = requests.get('https://api.iextrading.com/1.0/ref-data/symbols')
    stockList = r.json()
    return process.extractOne(text, stockList)[0]

In [6]:
company = getCompany(company_request)

In [7]:
symbol = company['symbol']
symbol

'CLBK'

In [8]:
company_name = company['name']

In [9]:
company_name

'COLUMBIA FINANCIAL INC'

In [10]:
data = yf.download(symbol, parse_dates=True, infer_datetime_format=True)

[*********************100%***********************]  1 of 1 downloaded


In [11]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-04-20,15.31,15.75,15.30,15.42,15.42,6816963
2018-04-23,15.40,15.50,15.35,15.45,15.45,1608100
2018-04-24,15.43,15.83,15.40,15.73,15.73,1057500
2018-04-25,15.50,15.70,15.50,15.56,15.56,566400
2018-04-26,15.68,15.73,15.50,15.72,15.72,789200
...,...,...,...,...,...,...
2019-12-02,16.70,16.79,16.29,16.34,16.34,183300
2019-12-03,16.33,16.59,16.23,16.54,16.54,206900
2019-12-04,16.56,16.76,16.52,16.57,16.57,132400


In [12]:
data['Daily Return'] = data['Close'].dropna().pct_change()
data

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2018-04-20,15.31,15.75,15.30,15.42,15.42,6816963,NaN
2018-04-23,15.40,15.50,15.35,15.45,15.45,1608100,0.001946
2018-04-24,15.43,15.83,15.40,15.73,15.73,1057500,0.018123
2018-04-25,15.50,15.70,15.50,15.56,15.56,566400,-0.010807
2018-04-26,15.68,15.73,15.50,15.72,15.72,789200,0.010283
...,...,...,...,...,...,...,...
2019-12-02,16.70,16.79,16.29,16.34,16.34,183300,-0.019208
2019-12-03,16.33,16.59,16.23,16.54,16.54,206900,0.012240
2019-12-04,16.56,16.76,16.52,16.57,16.57,132400,0.001814


In [13]:
# Set short and long windows
short_window = 5
long_window = 21

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['fast_close'] = data['Close'].ewm(halflife=short_window).mean()
data['slow_close'] = data['Close'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['fast_close'] > data['slow_close'], 1.0, 0.0)
data['crossover_short'] = np.where(data['fast_close'] < data['slow_close'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']

data.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return,fast_close,slow_close,crossover_long,crossover_short,crossover_signal
Date,,,,,,,,,,,,
2018-04-20,15.31,15.75,15.30,15.42,15.42,6816963,NaN,15.420000,15.420000,0.0,0.0,0.0
2018-04-23,15.40,15.50,15.35,15.45,15.45,1608100,0.001946,15.436038,15.435248,1.0,0.0,1.0
2018-04-24,15.43,15.83,15.40,15.73,15.73,1057500,0.018123,15.547878,15.536759,1.0,0.0,1.0
2018-04-25,15.50,15.70,15.50,15.56,15.56,566400,-0.010807,15.551565,15.542860,1.0,0.0,1.0
2018-04-26,15.68,15.73,15.50,15.72,15.72,789200,0.010283,15.595172,15.580664,1.0,0.0,1.0


In [14]:
# Plot the EMA of BTC/USD closing prices
#data[['Close', 'fast_close', 'slow_close']].plot(figsize=(20,10))

In [15]:
news_data = newsapi.get_everything(
    q=company_name or symbol, language="en", page_size=100
)

# Show the total number of news
news_data["totalResults"]

407

In [16]:
analyzer = SentimentIntensityAnalyzer()

In [17]:
# Define a function to get the sentiment scores
def get_sentiment_scores(text, date, source, url):
    sentiment_scores = {}

    # Sentiment scoring with VADER
    text_sentiment = analyzer.polarity_scores(text)
    sentiment_scores["date"] = date
    sentiment_scores["text"] = text
    sentiment_scores["source"] = source
    sentiment_scores["url"] = url
    sentiment_scores["compound"] = text_sentiment["compound"]
    sentiment_scores["pos"] = text_sentiment["pos"]
    sentiment_scores["neu"] = text_sentiment["neu"]
    sentiment_scores["neg"] = text_sentiment["neg"]
    if text_sentiment["compound"] >= 0.05:  # Positive
        sentiment_scores["normalized"] = 1
    elif text_sentiment["compound"] <= -0.05:  # Negative
        sentiment_scores["normalized"] = -1
    else:
        sentiment_scores["normalized"] = 0  # Neutral

    return sentiment_scores

In [18]:
# Empty list to store the DataFrame structure
sentiments_data = []

# Loop through all the news articles
for article in news_data["articles"]:
    try:
        # Get sentiment scoring using the get_sentiment_score() function
        sentiments_data.append(
            get_sentiment_scores(
                article["content"],
                article["publishedAt"][:10],
                article["source"]["name"],
                article["url"],
            )
        )

    except AttributeError:
        pass

# Create a DataFrame with the news articles' data and their sentiment scoring results
news_df = pd.DataFrame(sentiments_data)

# Sort the DataFrame rows by date
news_df = news_df.sort_values(by="date")

# Define the date column as the DataFrame's index
news_df.set_index("date", inplace=True)
news_df.head()


,text,source,url,compound,pos,neu,neg,normalized
date,,,,,,,,
2019-11-08,It is not uncommon to see companies perform we...,Yahoo.com,https://finance.yahoo.com/news/read-selling-in...,0.6486,0.174,0.778,0.048,1
2019-11-08,"""The names might not change, but the ownership...",Westword.com,https://www.westword.com/marijuana/denver-mari...,0.0000,0.000,1.000,0.000,0
2019-11-08,"Charlotte Nov 8, 2019 (Thomson StreetEvents) -...",Yahoo.com,https://finance.yahoo.com/news/edited-transcri...,0.0000,0.000,1.000,0.000,0
2019-11-08,"BOSTON(BUSINESS WIRE)Today, Columbia Seligman ...",Yahoo.com,https://finance.yahoo.com/news/columbia-seligm...,0.5859,0.121,0.879,0.000,1
2019-11-08,"BOSTON--(BUSINESS WIRE)--Today, Columbia Selig...",Businesswire.com,https://www.businesswire.com/news/home/2019110...,0.5859,0.124,0.876,0.000,1


In [19]:
news_df.tail(20)

,text,source,url,compound,pos,neu,neg,normalized
date,,,,,,,,
2019-12-03,"VANCOUVER, British Columbia--(BUSINESS WIRE)--...",Businesswire.com,https://www.businesswire.com/news/home/2019120...,0.4404,0.077,0.923,0.000,1
2019-12-03,"FAIR LAWN, N.J. and ROSELLE, N.J., Dec. 3, 201...",Yahoo.com,https://news.yahoo.com/columbia-financial-inc-...,0.4648,0.074,0.926,0.000,1
2019-12-03,"Throughout the past 10 years, Starion Energy h...",Yahoo.com,https://finance.yahoo.com/news/starion-energy-...,0.9517,0.396,0.604,0.000,1
2019-12-03,"HAMPTON, N.H., Dec. 3, 2019 /PRNewswire/ -- Pl...",Yahoo.com,https://news.yahoo.com/planet-fitness-complete...,0.2732,0.050,0.950,0.000,1
2019-12-04,"VANCOUVER, British Columbia, Dec. 4, 2019 /PRN...",Yahoo.com,https://news.yahoo.com/core-one-labs-announces...,0.0000,0.000,1.000,0.000,0
2019-12-04,"HAMPTON, N.H., Dec. 4, 2019 /PRNewswire/ -- Pl...",Yahoo.com,https://news.yahoo.com/planet-fitness-inc-anno...,0.8658,0.229,0.771,0.000,1
2019-12-04,"CALGARY, Alberta--(BUSINESS WIRE)--Olympia Fin...",Businesswire.com,https://www.businesswire.com/news/home/2019120...,0.5267,0.101,0.899,0.000,1
2019-12-04,When is a sale not a sale?\r\n1. When it hasn’...,Nakedcapitalism.com,https://www.nakedcapitalism.com/2019/12/calper...,-0.2732,0.073,0.811,0.116,-1
2019-12-05,The latest 13F reporting period has come and g...,Yahoo.com,https://finance.yahoo.com/news/hedge-funds-con...,0.3400,0.048,0.952,0.000,1


In [20]:
avg_sent_data = (
    news_df[["compound", "normalized"]].groupby(by=news_df.index).mean()
)
avg_sent_data.head()


,compound,normalized
date,,
2019-11-08,0.400071,0.714286
2019-11-09,-0.123983,-0.500000
2019-11-10,0.000000,0.000000
2019-11-11,0.397233,0.666667
2019-11-12,0.137775,0.125000


In [21]:
avg_sent_data

,compound,normalized
date,,
2019-11-08,0.400071,0.714286
2019-11-09,-0.123983,-0.500000
2019-11-10,0.000000,0.000000
2019-11-11,0.397233,0.666667
2019-11-12,0.137775,0.125000
2019-11-13,0.233320,0.400000
2019-11-14,0.494033,1.000000
2019-11-15,-0.043967,-0.166667
2019-11-16,0.335250,0.500000


In [22]:
signal_data =  pd.merge(data, avg_sent_data, left_index=True, right_index=True)

In [23]:
signal_data

,Open,High,Low,Close,Adj Close,Volume,Daily Return,fast_close,slow_close,crossover_long,crossover_short,crossover_signal,compound,normalized
2019-11-08,16.58,16.71,16.35,16.55,16.55,79000,-0.001809,16.518273,16.000197,1.0,0.0,1.0,0.400071,0.714286
2019-11-11,16.51,16.64,16.51,16.56,16.56,63100,0.000604,16.523675,16.018373,1.0,0.0,1.0,0.397233,0.666667
2019-11-12,16.56,16.67,16.41,16.60,16.60,54300,0.002415,16.533555,16.037257,1.0,0.0,1.0,0.137775,0.125000
2019-11-13,16.53,16.65,16.48,16.54,16.54,85300,-0.003614,16.534389,16.053581,1.0,0.0,1.0,0.233320,0.400000
2019-11-14,16.52,16.63,16.43,16.50,16.50,83200,-0.002418,16.529938,16.068075,1.0,0.0,1.0,0.494033,1.000000
2019-11-15,16.56,16.57,16.32,16.37,16.37,91700,-0.007879,16.509234,16.077878,1.0,0.0,1.0,-0.043967,-0.166667
2019-11-18,16.39,16.47,14.36,16.42,16.42,86000,0.003054,16.497683,16.088986,1.0,0.0,1.0,0.421500,1.000000
2019-11-19,16.44,16.62,16.42,16.56,16.56,146300,0.008526,16.505749,16.104279,1.0,0.0,1.0,0.501867,1.000000
2019-11-20,16.53,16.58,16.30,16.39,16.39,167400,-0.010266,16.490766,16.113556,1.0,0.0,1.0,0.073850,0.500000
2019-11-21,16.41,16.56,16.22,16.26,16.26,78200,-0.007932,16.460893,16.118311,1.0,0.0,1.0,0.159350,-0.250000
